In [ ]:
import numpy as np
import scipy.integrate

In [ ]:
#!f2py-3.5 -c -m dvrmod dvrmod.f90
import dvrmod

In [ ]:
from jftools import fedvr

In [ ]:
n = 51
f = lambda x: x**3 - 2*x**2
t, w = fedvr.simpsrule(n)
print(f(t) @ w, scipy.integrate.simps(f(t),t))

In [ ]:
print(fedvr.gbslve(1,t,w) == dvrmod.helpers.gbslve(1,t,w))
%timeit -n100 dvrmod.helpers.gbslve(1,t,w)
%timeit -n100 fedvr.gbslve(1,t,w)

In [ ]:
n = 15
endpts = np.r_[-1.,1.]
t1, w1 = dvrmod.helpers.gaussq('legendre',0,0,len(endpts),endpts,np.empty(n))
t2, w2 = fedvr.gaussq('legendre',n,endpts)
assert np.allclose(t1,t2)
assert np.allclose(w1,w2)

%timeit -n100 t1, w1 = dvrmod.helpers.gaussq('legendre',0,0,len(endpts),endpts,np.empty(n))
%timeit -n100 t2, w2 = fedvr.gaussq('legendre',n,endpts)

In [ ]:
A = np.random.random((2,n))
t2, w2 = scipy.linalg.eig_banded(A)
d, e = A[1], np.r_[A[0,1:],0.]

t1 = A[1,:]
ee = np.r_[A[0,1:],0.]
w1 = np.zeros(n); w1[0]=1.
ierr = dvrmod.helpers.gbtql2(t1,ee,w1)
assert ierr==0
assert np.allclose(t1,t2)
assert np.allclose(abs(w2[0]),abs(w1))

%timeit -n500 w1.copy()
%timeit -n500 t2, w2 = scipy.linalg.eig_banded(A)
%timeit -n500 ierr = dvrmod.helpers.gbtql2(t1.copy(),ee.copy(),w1.copy())

In [ ]:
def checkgaussq(kind,n,alpha,beta,endpts):
    fendpts = np.zeros(2)
    fendpts[:len(endpts)] = endpts
    t1, w1 = dvrmod.helpers.gaussq(kind,alpha,beta,len(endpts),fendpts,np.empty(n))
    t2, w2 = fedvr.gaussq(kind,n,endpts,alpha,beta)
    assert np.allclose(t1,t2)
    assert np.allclose(w1,w2)

for n in [12,19,28,55]:
    kinds = ['legendre','chebyshev-1','chebyshev-2','hermite','jacobi','laguerre']
    if n%2==1:
        kinds.append('simpson')
    for kind in kinds:
        checkgaussq(kind,n,0,0,[])
    for alpha in [-0.99,-0.5,0,0.31351,0.5,1,2]:
        checkgaussq('laguerre',n,alpha,0,[])
        for beta in [-0.99,-0.5,0,0.31351,0.5,1,2]:
            checkgaussq('jacobi',n,alpha,beta,[])

In [ ]:
x = np.linspace(0,2,13)
y = np.linspace(0.01,1,27)

fres = dvrmod.helpers.lgngr(x,x,'a')
pres = fedvr.lgngr(x,x)
print(*(np.allclose(f,p) for f,p in zip(fres,pres)))

fres = dvrmod.helpers.lgngr(x,y,'a')
pres = fedvr.lgngr(x,y)
print(*(np.allclose(f,p) for f,p in zip(fres,pres)))

In [ ]:
def dvr_reg(*args,**kwargs):
    r = fedvr.fedvr_region(*args,**kwargs)
    return r.x, r.wt, r.dx, r.dx2, r.ke

In [ ]:
fres = dvrmod.dvr_reg(18,[-3,10])
pres =  dvr_reg(18,[-3,10])
print(*(np.allclose(f,p) for f,p in zip(fres,pres)))

In [ ]:
%timeit -n100 fres = dvrmod.dvr_reg(15,[-3,10])
%timeit -n100 pres = dvr_reg(15,[-3,10])